# Top 10 Authors by Book Count
Visualizing author popularity from Open Library Harry Potter search results using ibis and dlt

## Import Dependencies

In [ ]:
import ibis
import ibis.expr.types as ibis_types
import plotly.express as px
import pandas as pd

## Connect to dlt Dataset via Ibis

In [ ]:
# Connect to the dlt-managed DuckDB database
con = ibis.duckdb.connect('open_library_pipeline.duckdb')

# List available tables in the open_library_data dataset
tables = con.list_tables(database='open_library_data')
print(f"Available tables: {tables}")

## Load the Author Data

In [ ]:
# Access the authors table from the dlt pipeline
authors_table = con.table('books__author_name', database='open_library_data')

# Inspect the table structure
print(authors_table.schema())

## Query Top 10 Authors

In [ ]:
# Get the top 10 authors by book count
top_authors = (
    authors_table
    .group_by('value')
    .aggregate(book_count=authors_table.value.count())
    .order_by(ibis.desc('book_count'))
    .limit(10)
    .execute()
)

# Rename columns for clarity
top_authors = top_authors.rename(columns={'value': 'author_name'})

print(top_authors)

## Visualize Top Authors

In [ ]:
# Create an interactive bar chart
fig = px.bar(
    top_authors,
    x='author_name',
    y='book_count',
    title='Top 10 Authors by Book Count (Harry Potter Search)',
    labels={'author_name': 'Author Name', 'book_count': 'Number of Books'},
    color='book_count',
    color_continuous_scale='viridis',
    text='book_count'
)

# Customize layout
fig.update_layout(
    xaxis_tickangle=-45,
    height=500,
    showlegend=False,
    hovermode='x unified'
)

fig.update_traces(textposition='outside')
fig.show()

## Summary Statistics

In [ ]:
print(f"Total authors in top 10: {len(top_authors)}")
print(f"Average books per top author: {top_authors['book_count'].mean():.1f}")
print(f"Most prolific author: {top_authors.iloc[0]['author_name']} with {top_authors.iloc[0]['book_count']} books")